In [1]:
# set up the notebook

%load_ext autoreload
%autoreload 2

import sys
import os
import logging
logging.basicConfig(level=logging.ERROR)

# Preparation

This part will help you get access to OpenAI API and run your first completion.
- For more information about model size and pricing, see [this page](https://openai.com/api/pricing/).
- For a complete explanation of models, please see [the model overview](https://platform.openai.com/docs/models/overview).
- For a detailed API walkthrough (with examples), see [this documentation page](https://beta.openai.com/docs/api-reference/completions).

In [2]:
# get openAI access and see its model list

import openai
import json

with open("./credential.json", "r") as f:
    credential = json.load(f)["openai"]
openai.api_key = credential

# Test to see if you've got the access successfuly using the cheapest model.
completion = openai.Completion.create(
  model="ada",
  prompt="Say this is a test",
  max_tokens=7,
  temperature=0
)
completion

<OpenAIObject text_completion id=cmpl-6t5LX0pEP5fUsU2E3cTg0rsY6OhJh at 0x7fea515ad9b0> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " of your ability to read the signs"
    }
  ],
  "created": 1678586611,
  "id": "cmpl-6t5LX0pEP5fUsU2E3cTg0rsY6OhJh",
  "model": "ada",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 7,
    "prompt_tokens": 5,
    "total_tokens": 12
  }
}

In [3]:
# If you got the completion successfully, you should be able to access the generated text through this line:
completion["choices"][0]["text"]

' of your ability to read the signs'

# Writeup 0: Task and Strategy Description

Here, you should describe what task you are working on, and what workflow/pipeline you intend to replicate (from which crowdsourcing paper). As a reminder, you should [pick a crowdsourcing paper here](https://docs.google.com/spreadsheets/d/1nIoU04CulTH128-r6rtykhuqNdd37UJXShw2J19IEdE/edit?usp=sharing). The spreadsheet also points to example tasks in the crowdsourcing papers; However, you DON'T have to stick to the paper-provided input/output. Please feel free to come up with your own tasks as long as they seem suitable for the paper/pipeline you are replicating.


**EDIT THIS PART TO PROVIDE AN OVERVIEW OF YOUR ATTEMPTS**.

- **Task Description**: Create three metaphors for a given concept, so that we can explain the different aspects of crowdsourcing in a poetic way. A metaphor may look like:

    > In crowdsourcing, people are like bees; they work together to make honey.

    With the concept being “crowdsourcing”, the simile being “bees”, and the similar aspect being “work together.”
    
    This task is from paper [AI Chains: Transparent and Controllable Human-AI Interaction by Chaining Large Language Model Prompts](https://arxiv.org/pdf/2110.01691.pdf), Case 0 in Appendix B.3.
- **Example Input/output**: Write >=3 input-output pairs of your task. You should test your strategy on all the three examples.
    ```
    Input: Crowdsourcing
    Output: 
    1. crowdsourcing is like a team sport in that it brings people to achieve one goal.
    2. Crowdsourcing is like a vast, open-source library, with everyone contributing to the shelves of knowledge.
    3. Crowdsourcing was like a quilt, with each individual contributing a patch of fabric to create a larger piece of art.

    Input: Gratitude
    Output:
    1. gratitude is like a stream in that it’s a force that can carry you along.
    2. Gratitude is like a generous river that overflows with blessings.
    3. Gratitude is a feather in the cap of humility.

    Input: loss
    Output:
    1. loss is like a wing in that it’s something you never wanted to lose, and it can take you away.
    2. The loss was like a dark cloud of disappointment hovering over me.
    3. Loss is a heavy burden, weighing down the heart like a funeral shroud.
    ```
- **Workflow prompting strategy**: The pipeline helps further specify an abstract query into more specific aspects. Given an input concept, we will
    - First find three unique traits of the concept
    - Ask the model to write a different metaphor for each trait.
    
    For example:
    ```
    INPUT: gratitude
    Output: 
    TRAIT => OUTPUT: 
    Appreciation => Gratitude is a warm embrace, showing appreciation for all that is given.
    Generosity => Gratitude is like a generous river that overflows with blessings.
    ```
- **Crowdsourcing paper**: [Towards Large-Scale Collaborative Planning: Answering High-Level Search Queries Using Human Computation](https://ojs.aaai.org/index.php/AAAI/article/view/8092) -- where the pipelining strategy comes from.

## Coding: Use this part to do your prompting.

**Please use `text-davinci-003` when you generate your final model outputs.** `text-davinci-003` is effectively InstructGPT, and it can do any language task with better quality, longer output, and consistent instruction-following than other versions of GPT-3 ([as discussed in this page](https://platform.openai.com/docs/models/overview)). Using the latest model can give you a better idea on what these models are (not) capable of.

Tips for prompting:
- [OpenAI playground](https://beta.openai.com/playground) offers more interactive prompting experiments.
- Read the [API documentation](https://beta.openai.com/docs/api-reference/completions?lang=python) to use the right parameters (e.g., `temperature`, `max_length`).
- Read the [tutorial doc](https://beta.openai.com/docs/introduction) to get a general sense of GPT-3.
- Read [OpenAI examples](https://beta.openai.com/examples) to get a sense of what GPT-3 can do.

Tips for saving credits:

- You can use a cheaper model for tweaking your prompts (see [pricing](https://openai.com/api/pricing/)) before generating the final responses using the most capable model `text-davinci-003`. However, be careful that sometimes a prompt that seems to not work for any other models may works for `text-davinci-003`.
- [AI21](https://www.ai21.com/studio) provides a model similar to GPT-3 / API service similar to OpenAI, and it involves more free credits for you to try. If you choose to use this, remember to also get its API key and set up the credential.
- Play with [ChatGPT](https://chat.openai.com/chat) which is a free and similarly capable model. It uses a dialog interface (vs. InstructGPT / `text-davinci-003` uses text completion interface), so naturally your way of interacting with the model may change a bit (e.g., you will ask questions there but use incomplete sentences here.)

## Baseline prompting

Use this section to do a one-step prompting for your selected task.

In [13]:
# Write your code here.
# If needed, change model="text-davinci-003" to cheaper models!
def baseline_prompt(concept, model="text-davinci-003", max_tokens=100, temperature=0.7, top_p=1):
    prompt = f"""The following is a list of three metaphors on '{concept}'.
Concept: {concept}
Metaphors: 1."""
    completion = openai.Completion.create(
        model=model,
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p
    )
    output = completion["choices"][0]["text"].split("4.")[0]
    print(f"INPUT: {concept}")
    print(f"OUTPUT: {output}")
    print()
    
baseline_prompt("crowdsourcing")

baseline_prompt("gratitude")

baseline_prompt("loss")

INPUT: crowdsourcing
OUTPUT:  A quilt made of many pieces; 2. A hive of creativity; 3. A river of collective knowledge.

INPUT: gratitude
OUTPUT:  Gratitude is a garden of blessings. 
2. Gratitude is a river of joy. 
3. Gratitude is a sunbeam of appreciation.

INPUT: loss
OUTPUT:  His heart was an empty void. 
2. The grief was a heavy weight on his shoulders. 
3. He felt like he was walking through a dark tunnel.



## Crowdsourcing-Pipeline-inspired prompting

Use this section to your selected pipeline prompting for your task.

In [12]:
# Write your code here.
from functools import partial
import re

def pipeline_prompt(concept, model="text-davinci-003", max_tokens=100, temperature=0.7, top_p=1):
    prompt_concept_to_trait = f"""The following three phrases describes three different and unique traits of the given concept.
Concept: {concept}
Unique traits: 1."""
    completion = openai.Completion.create(
        model=model,
        prompt=prompt_concept_to_trait,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p
    )
    traits = completion["choices"][0]["text"].split("4.")[0]
    traits = [t.strip() for t in re.split(r'\d+\.', traits) if t.strip()]
    
    metaphors = []
    for trait in traits:
        prompt_trait_to_metaphor = f"""Given the concept and its unique trait, write a metaphor that conveys the corresponding trait.
Concept: {concept}
trait: {trait}
metaphor:"""
        completion = openai.Completion.create(
            model=model,
            prompt=prompt_trait_to_metaphor,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p
        )
        metaphors.append(completion["choices"][0]["text"].strip())
    metaphor = '\n'.join(metaphors)
    print(f"INPUT: {concept}")
    print(f"TRAITS: \n{traits}")
    print(f"OUTPUT: \n{metaphor}")
    print()

pipeline_prompt("crowdsourcing")

pipeline_prompt("gratitude")

pipeline_prompt("loss")



INPUT: crowdsourcing
TRAITS: 
['Open source collaboration', 'Leveraging collective intelligence', 'Collaborative problem-solving.']
OUTPUT: 
Crowdsourcing is like a quilt, with each individual contributing a unique square to create a beautiful, shared piece of art.
Crowdsourcing is like a hive of minds, collectively buzzing with creative solutions.
Crowdsourcing is like a group of puzzle pieces coming together to form the perfect picture.

INPUT: gratitude
TRAITS: 
['Appreciation of kindness', 'Acknowledgement of help', 'Expression of thankfulness.']
OUTPUT: 
Gratitude is like a rose, blossoming in appreciation of the kindness bestowed upon it.
Gratitude is a bright beacon, illuminating the path of kindness and acknowledging the help of others.
Gratitude is a sunflower, its vibrant petals reaching up to express its thankfulness.

INPUT: loss
TRAITS: 
['Desolation,', 'Disappointment,', 'Detachment.']
OUTPUT: 
The world was a barren desert, devoid of hope and full of desolation.
Disappoi

# Writeup 1: Report & Reflection

Fill in the following three sections by reflecting on your results.

## Report the result

For all the inputs you tried, summarized the input, baseline output, pipeline output, which output you like and why. To answer "why", you should first think of some criteria you want to use for evaluating the output:

### Important criteria:
1. Diversity: The output should be diverse.
2. Clarity: The output should be descriptive and clearly explains why a metaphor is used.

### Inputs and outputs:

- **Input**: loss
- **Baseline output**: (diversity: 5/5, clarity: 1/5)
    - His heart was an empty void. 
    - The grief was a heavy weight on his shoulders. 
    - He felt like he was walking through a dark tunnel.

- **Pipeline output**: (diversity: 5/5, clarity: 1/5)
    - The world was a barren desert, devoid of hope and full of desolation.
    - Disappointment settled over the room like a thick fog, obscuring the joy that had been there before.
    - The grief of loss was like a balloon set free, slowly fading away in the distance.

- **Which output you like and why**:
    I like the pipeline output, because it's much more clear why a simile is picked.

--- 

- **Input**:
- **Baseline output**:
- **Pipeline output**:
- **Which output you like and why**:


--- 

...

## Reflect on prompting effectiveness

Write some paragraphs to describe: Did you find the pipeline prompting workflow effective? Why or Why not?

## Envision possible improvements

Write some paragraphs to describe: What are some possible ways to further improve the pipeline or prompt design?

# Writeup 2 (Optional): InstructGPT vs. ChatGPT

If you also tried to use the [ChatGPT](https://chat.openai.com/chat) interface to complete your task, you might have thoughts on these following two questions!

## Chat interface vs. complete-the-sentence interface
Did your way of prompting change based on the interaction interface? What do you think are some pros and cons for the chat interface (you used in ChatGPT), and the complete-the-sentence interface (you used in this notebook)?

## Qualitative reflection on outputs
Provide some examples of your ChatGPT output. Did you notice any differences?


- **Input**:
- **InstructGPT output**:
- **ChatGPT output**:
- **Which output you like and why**:

---

- **Input**:
- **InstructGPT output**:
- **ChatGPT output**:
- **Which output you like and why**: